In [31]:
import nltk
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from itertools import tee, islice, chain
import wikipedia
from nltk.corpus import sentiwordnet as swn
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#nltk.download('movie_reviews')
#nltk.download('punkt')


**Import necessary libraries**

In [32]:
def getVikipedia():
    
    text=wikipedia.page("Superman").content
    text2=wikipedia.page("Rocky").content
    text3=wikipedia.page("The Piano").content
    text4=wikipedia.page("Titanic").content
    text5=wikipedia.page("Inception").content
    text6=wikipedia.page("Die Hard").content
    text7=wikipedia.page("Godfather").content
    text8=wikipedia.page("Tokyo Story").content
    
    
    texts=text+text2+text3+text4+text5+text6+text7+text8
    
    array=texts.split()

    return array

def searchWord():
    
    array=getVikipedia()
    dataset=[]
    for i in array:
        if(compareWithTxt(i)==True):
            dataset.append(i)
    return dataset


def compareWithTxt(word):
    
    paths=['/Users/ertanuysal/Downloads/2020-assignment_2 (4) 2/100-400.txt',
           '/Users/ertanuysal/Downloads/2020-assignment_2 (4) 2/400-700.txt']
    
    for i in paths:
    
        f = open(i)

        # use readline() to read the first line 
        line = f.readline()

        while line:

            wordsOfLine=line.split()

            if(word.upper() == wordsOfLine[0]):
                return True

            line = f.readline() # use realine() to read next line
        f.close()
    return False

**Aim of this code block is getting data as word form in wikipedi and checking whether it is in txt files or not.**

<h3>getVikipedia:</h3> Get vikipedi articles context with word form
    
<h3>searchWord:</h3> If the word is in the txt files ,add into dataset list.
    
<h3>compareWithTxt:</h3> Check whether word is in the txt files or not.
    



In [33]:





def  checkStrongOrWeak(word):

    f = open('/Users/ertanuysal/Desktop/ML_HW2_250201064/subjclueslen1-HLTEMNLP05.txt')
    # use readline() to read the first line 
    line = f.readline()

    while line:

        
        wordsOfLine=line.split()
        nameOfLine=wordsOfLine[2].split("=")
        
        if(nameOfLine[1]==word):
            types=wordsOfLine[0].split("=")
            
            if(types[1]=="strongsubj"):
                return True
            else:   
                return False
              
        line = f.readline() # use realine() to read next line
    f.close()

def get_pos(string):
    string = nltk.word_tokenize(string)
    pos_string = nltk.pos_tag(string)
    return pos_string [0][1]


def pozOrNeg(word):
    
    sent          = TextBlob(word)
    sent          = TextBlob(word, analyzer = NaiveBayesAnalyzer())
    classification= sent.sentiment.classification
    if (classification=="pos"):
        return True
    else:
        return False
    
def score(word):
    
    good = swn.senti_synsets(word, 'n')
    posscore=0
    negscore=0
    for synst in good:

        posscore=posscore+synst.pos_score()
        negscore=negscore+synst.neg_score()
    if(posscore>negscore):
        return True
    if(negscore>posscore):
        return False





def findPoint(word):
    paths=['/Users/ertanuysal/Downloads/2020-assignment_2 (4) 2/100-400.txt',
           '/Users/ertanuysal/Downloads/2020-assignment_2 (4) 2/400-700.txt']
    
    for i in paths:
        f = open(i)

        # use readline() to read the first line 
        line = f.readline()

        while line:
            wordsOfLine=line.split()
            if(wordsOfLine[0]==word.upper()):
                
                score=wordsOfLine[1]
                score=int(score)
                print(score)

                if(score>323):
                    return True

            line = f.readline() # use realine() to read next line
        f.close()
    return False
    



      



**Aim of this code block is create for features for each word.**

<h3>checkStrongOrWeak:</h3> Check the word is strongword or weakword according to subjclueslen1-HLTEMNLP05.txt file.

<h3>get_pos:</h3> Get the part of speach tag of word

<h3>pozOrNeg:</h3> Check the word is pozitif or negatif word.

<h3>score:</h3> Check the word is pozitivie or negative word.(i used this method in this assinghment because it is faster)

<h3>findPoint:</h3> Defines the label of word as abstract or concrete with threshold(we determined threshold value as 323 to ensure a balanced distribution for labels)



In [34]:
rows=[]       
def mainFunction(dataset):
    
    twoBefore,previous,next_,twoAfter,current= None,None,None,None,None
    l = len(dataset)
    

    for index, obj in enumerate(dataset):
        current=dataset[index]
        if index > 0:
            previous = dataset[index - 1]
        if index < (l - 1):
            next_ = dataset[index + 1]

        if index > 0:
            twoBefore = dataset[index - 2]
        if index < (l - 2):
            twoAfter = dataset[index + 2]

        checkStrong=checkStrongOrWeak(current)
        checkWeak= not checkStrong
        pozitivity=score(current)
        
        
        
        if(twoBefore and previous and next_ and twoAfter and current and checkStrong and pozitivity != None):
        
            print(twoBefore)
            print(previous)
            print(current)
            print(next_)
            print(twoAfter)

            pos=get_pos(current)
            pos_before=get_pos(previous)
            pos_2before=get_pos(twoBefore)
            pos_after=get_pos(next_)
            pos_2after=get_pos(twoAfter)

            
            pozitivity=score(current)
            negativity= not pozitivity
            
            pozitivity_after=score(next_)

            
            pozitivity_before=score(previous)
            
            
            
            if(pozitivity_before==None):
                negativity_before=None
            else:  
                negativity_before=not pozitivity_before
                
            point=findPoint(current)

        

            row=[current ,str(pos) , str(pos_before), str(pos_2before) ,str(pos_after), str(pos_2after) , str(checkStrong) ,str(checkWeak) , str(pozitivity),str(negativity) , str(pozitivity_after), str(negativity_before) ,str(point)]
            rows.append(row)
            
    print(rows)
    
        #with open('/Users/ertanuysal/Desktop/ML_HW1_MATERIAL/test.csv', 'w') as f:
            #for line in rows:
                #f.write(line)
                #f.write("\n")
            #f.close()
        
        
        
        
   
        
            
        
        
        
        
#test=["champion","hate","door","test","snake","no","nine","knisd","yes","ring","wrong","happy","dsad"]           
            

            
#print(searchWord())
print(mainFunction(searchWord()))

#print(getVikipedia())   
#print(compareWithTxt())
#print(checkStrongOrWeak())
#print(get_pos())
#print(score("other"))
#print(pozOrNeg("Web"))


the
his
love
interest
and
311
over
their
love
of
to
311
is
his
will
by
an
275
was
a
hero
of
a
428
and
the
opportunity
to
present
306
that
he
must
his
for
297
to
a
star
football
The
574
not
two
worth
of
on
257


KeyboardInterrupt: 

The purpose of this block of code is to create a row-by-row dataset by determining the properties of all words.

If words do not have pozitivity value , pos tag value with the nearest words and we can not check whether it is strong or not, we removed word in the dataset.(postag,pozitivity-negativity,strongOrWeak features are required)

**This code block takes about 6 minutes to run, as it is examined whether each word provides the condition.**


In [ ]:
import pandas as pd
import numpy as np
columns = ['word','pos','pos_before','pos_2before','pos_after','pos_2after','checkStrong','checkWeak','pozitivity','negativity','pozitivity_after','negativity_before','label']
# load dataset
data=pd.DataFrame(rows,columns=columns)
#data = pd.read_csv("/Users/ertanuysal/Desktop/ML_HW1_MATERIAL/test.csv", header=None, names=col_names)
data.head()


**This block of code shows part of the dataset with the column names.**


In [ ]:
for column in data.columns:
    if data[column].dtype == type(object) or data[column].dtype == type(bool) :
        le = preprocessing.LabelEncoder()
        data[column] = le.fit_transform(data[column])
        

**Type conversion is made to run logistic regression function**

In [ ]:
features = ['word','pos', 'pos_before', 'pos_2before','pos_after','pos_2after','checkStrong','checkWeak','pozitivity','negativity',
                'pozitivity_after','negativity_before']
X = data[features] # Features
y = data.label 

**Features to be used in logistic regression and target variable are determined**



In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=0)

** In the dataset,There are 303 words which satisfy the conditions.We allocated %70 of 303 words in train data **

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train,y_train) # training part
y_pred=logreg.predict(X_test) #prediction part on test values.



In [ ]:
def calculateAcc(y_pred,y_test):
    correct=0
    for index,item in enumerate(y_pred):
    
        if(item==y_test.to_numpy()[index]):
            
            correct+=1
    acc=correct/len(y_test)
    return acc
calculateAcc(y_pred,y_test)



    



<h3>calculateAcc:</h3> This method compare each values of y_pred and y_test.If values are same,It means that we predict correctly for that value and increases counter as one.

As a result we can obtain accuracy.



In [ ]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
print(cnf_matrix)

Binary confustion matrix of test values.


